In [ ]:
!pip install -q tensorflow torch numpy matplotlib torchvision scikit-learn seaborn

: 

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ TensorFlow encontrou {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  - {gpu.name}")
    
    # Verifica se uma operação simples é alocada na GPU
    try:
        print("\nAttempting a test operation on the GPU...")
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            c = tf.matmul(a, b)
        print("✅ Teste de operação na GPU foi bem-sucedido.")
        print("✅ Isso confirma que o TensorFlow pode usar a GPU para os cálculos.")
        print("Se o treino ainda estiver lento, o problema pode ser um gargalo de dados (pouco provável com MNIST) ou outra configuração.")

    except RuntimeError as e:
        print("\n❌ Ocorreu um erro durante o teste de operação na GPU:")
        print(e)
        print("\n❌ Isso sugere um problema de compatibilidade com CUDA/CuDNN dentro do ambiente conda.")

else:
    print("❌ ERRO CRÍTICO: TensorFlow não encontrou nenhuma GPU.")
    print("❌ O treinamento será executado na CPU, o que explica a lentidão extrema.")
    print("Por favor, verifique a instalação dos drivers da NVIDIA e do ambiente conda.")

In [ ]:
import sys
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# --- CORREÇÃO DE CAMINHO ---
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# --- OTIMIZAÇÃO DE MEMÓRIA DA GPU ---
#gpus = tf.config.experimental.list_physical_devices('GPU')
#if gpus:
#  try:
#    for gpu in gpus:
#      tf.config.experimental.set_memory_growth(gpu, True)
#    print(f"Otimização de memória (Memory Growth) ativada para {len(gpus)} GPU(s).")
#  except RuntimeError as e:
#    print(e)

# --- IMPORTS DO PROJETO ---
from config.training_config import CosFaceConfig
from src.backbones import vgg8
from src.optimizers.scheduler import CosineAnnealingScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, TerminateOnNaN
from tensorflow.keras.metrics import Precision, Recall

print("Setup completo. Módulos e configurações carregados.")

In [ ]:
# Célula 2: Definição dos Parâmetros do Experimento
class Args:
    arch = 'vgg8_cosface'
    num_features = 3
    scheduler = 'CosineAnnealing'
    epochs = 50
    batch_size = 64
    optimizer = 'SGD'
    lr = 0.5
    min_lr = 0.005
    momentum = 0.5
    name = f'mnist_{arch}_{num_features}d'

args = Args()


In [ ]:
# Célula 3: Preparação de Pastas e Dados
checkpoint_dir = f'../experiments/checkpoints/{args.name}'
log_path = f'../experiments/logs/{args.name}_log.csv'

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs('../experiments/logs/', exist_ok=True)

print('Config -----')
for arg in vars(args):
    print(f'{arg}: {getattr(args, arg)}')
print('------------')

(X, y), (X_test, y_test) = mnist.load_data()
X = X[:, :, :, tf.newaxis].astype('float32') / 255
X_test = X_test[:, :, :, tf.newaxis].astype('float32') / 255
y = tf.keras.utils.to_categorical(y, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [ ]:
def plot_training_history(history, save_path):
    """Plota e salva as curvas de acurácia e perda."""
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Encontra dinamicamente as chaves corretas
    acc_key = next((k for k in history.history.keys() if 'accuracy' in k and 'val' not in k), 'accuracy')
    val_acc_key = next((k for k in history.history.keys() if 'accuracy' in k and 'val' in key), 'val_accuracy')
    
    # Plot de Acurácia
    ax1.plot(history.history[acc_key], label='Treino Acurácia')
    ax1.plot(history.history[val_acc_key], label='Validação Acurácia')
    ax1.set_title('Histórico de Acurácia')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Acurácia')
    ax1.legend()
    
    # Plot de Perda
    ax2.plot(history.history['loss'], label='Treino Perda')
    ax2.plot(history.history['val_loss'], label='Validação Perda')
    ax2.set_title('Histórico de Perda')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Perda')
    ax2.legend()
    
    fig.tight_layout()
    plt.savefig(os.path.join(save_path, 'training_history.png'))
    plt.show()

def plot_confusion_matrix(model, X_test, y_test_cat, save_path):
    """Plota e salva a matriz de confusão."""
    y_pred_probs = model.predict([X_test, y_test_cat])
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test_cat, axis=1)
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusão')
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.savefig(os.path.join(save_path, 'confusion_matrix.png'))
    plt.show()

def plot_learning_rate(history, save_path):
    """Plota e salva o histórico da taxa de aprendizado."""
    if 'lr' in history.history:
        plt.figure(figsize=(10, 5))
        plt.plot(history.history['lr'], label='Taxa de Aprendizagem')
        plt.title('Agendamento da Taxa de Aprendizagem')
        plt.xlabel('Época')
        plt.ylabel('Learning Rate')
        plt.legend()
        plt.savefig(os.path.join(save_path, 'learning_rate_schedule.png'))
        plt.show()

print("Funções de plotagem prontas.")

In [ ]:
# Carrega a configuração
config = CosFaceConfig()
FIGURES_PATH = 'reports/figures'

# Prepara pastas
os.makedirs(os.path.dirname(config.CHECKPOINT_PATH), exist_ok=True)
os.makedirs(os.path.dirname(config.LOG_PATH), exist_ok=True)
os.makedirs(FIGURES_PATH, exist_ok=True)

# Carrega Dados
(X, y), (X_test, y_test) = mnist.load_data()
X = X[..., tf.newaxis].astype('float32') / 255
X_test = X_test[..., tf.newaxis].astype('float32') / 255
y_cat = tf.keras.utils.to_categorical(y, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

# Compila Modelo
optimizer = SGD(learning_rate=config.LEARNING_RATE, momentum=config.MOMENTUM)
model = getattr(vgg8, config.ARCH)(config)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])

# Callbacks
model_checkpoint = ModelCheckpoint(config.CHECKPOINT_PATH, verbose=1, save_best_only=True)
weights_checkpoint_path = config.CHECKPOINT_PATH.replace('.keras', '.weights.h5')
weights_checkpoint = ModelCheckpoint(weights_checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)
callbacks = [model_checkpoint, weights_checkpoint, CSVLogger(config.LOG_PATH), TerminateOnNaN()]
if config.SCHEDULER == 'CosineAnnealing':
    callbacks.append(CosineAnnealingScheduler(T_max=config.EPOCHS, eta_max=config.LEARNING_RATE, eta_min=config.MIN_LEARNING_RATE, verbose=1))

# Treinamento
history = model.fit([X, y_cat], y_cat, validation_data=([X_test, y_test_cat], y_test_cat),
                    batch_size=config.BATCH_SIZE,
                    epochs=config.EPOCHS,
                    callbacks=callbacks,
                    verbose=1)

print("\n--- Treinamento Concluído ---")

In [ ]:
print("\n--- Iniciando Análise Pós-Treino ---")
# Carrega o melhor modelo salvo para a análise
model.load_weights(weights_checkpoint_path)

# Gera os gráficos
plot_training_history(history, FIGURES_PATH)
plot_confusion_matrix(model, X_test, y_test_cat, FIGURES_PATH)
plot_learning_rate(history, FIGURES_PATH)

print("Análise concluída. Gráficos salvos em 'reports/figures/'.")